In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import datetime
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from Clean import chennai_house_df
import sklearn

In [108]:
df = chennai_house_df.copy()

In [ ]:
# Select required columns

df = df[['AREA','BUILDING_AGE', 'INT_SQFT', 'DATE_SALE', 'DIST_MAINROAD', 'N_BEDROOM',
       'N_BATHROOM', 'N_ROOM', 'SALE_COND', 'PARK_FACIL', 'DATE_BUILD',
       'BUILDTYPE', 'UTILITY_AVAIL', 'STREET', 'MZZONE', 'QS_ROOMS',
       'QS_BATHROOM', 'QS_BEDROOM', 'QS_OVERALL', 'SALES_PRICE']]

## Categorical column Target encoding

In [ ]:
# Filter out categorical column
X_cat = df.select_dtypes('object')
X_cat

,AREA,SALE_COND,PARK_FACIL,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE
0,Karapakkam,AbNormal,Yes,Commercial,AllPub,Paved,A
1,Anna Nagar,AbNormal,No,Commercial,AllPub,Gravel,RH
2,Adayar,AbNormal,Yes,Commercial,ELO,Gravel,RL
3,Velachery,Family,No,Others,NoSewr,Paved,I
4,Karapakkam,AbNormal,Yes,Others,AllPub,Gravel,C
...,...,...,...,...,...,...,...
7098,Karapakkam,AdjLand,No,Others,ELO,No Access,RM
7099,Velachery,Family,Yes,Others,NoSeWa,No Access,RH
7100,Velachery,Normal Sale,No,House,NoSeWa,Gravel,I
7101,Karapakkam,Partial,Yes,Commercial,ELO,Paved,RL


In [140]:
order_col = [list(df.groupby(i)['SALES_PRICE'].mean().sort_values(ascending=False).index) for i in X_cat.columns ]

In [ ]:
# encode the columns
from sklearn.preprocessing import OrdinalEncoder
order = OrdinalEncoder(categories=order_col,dtype='int64')
X_cat_En = pd.DataFrame(order.fit_transform(X_cat),columns=order.get_feature_names_out())
X_cat_En

,AREA,SALE_COND,PARK_FACIL,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE
0,6,2,0,0,0,1,5
1,1,2,1,0,0,0,2
2,5,2,0,0,3,0,1
3,3,3,1,1,1,1,3
4,6,2,0,1,0,0,4
...,...,...,...,...,...,...,...
7098,6,0,1,1,3,2,0
7099,3,3,0,1,2,2,2
7100,3,1,1,2,2,0,3
7101,6,4,0,0,3,1,1


In [ ]:
# separate 
X_cat_En1 = df[['N_BEDROOM','N_BATHROOM','N_ROOM']]
X_cat_En1

,N_BEDROOM,N_BATHROOM,N_ROOM
0,1.0,1.0,3
1,2.0,1.0,5
2,1.0,1.0,3
3,3.0,2.0,5
4,1.0,1.0,3
...,...,...,...
7098,1.0,1.0,2
7099,3.0,2.0,5
7100,2.0,1.0,4
7101,1.0,1.0,2


In [154]:
y = df['SALES_PRICE']

Numerical columns encoding using standardization

In [157]:
X_num = df.select_dtypes(['int32','int64','float'])
X_num.drop(columns=['N_BEDROOM','N_BATHROOM','N_ROOM'],inplace=True)

In [161]:
X_num.corr()['SALES_PRICE'].sort_values(ascending=False)

SALES_PRICE      1.000000
INT_SQFT         0.612026
QS_ROOMS         0.021960
QS_OVERALL       0.019784
DIST_MAINROAD    0.018679
QS_BEDROOM       0.018430
QS_BATHROOM     -0.011361
BUILDING_AGE    -0.118644
Name: SALES_PRICE, dtype: float64

In [163]:
X_num = X_num[['INT_SQFT','BUILDING_AGE']]

In [167]:
from sklearn.preprocessing import StandardScaler
Scale = StandardScaler()
X_num_en = pd.DataFrame(Scale.fit_transform(X_num),columns=Scale.get_feature_names_out())
X_num_en

,INT_SQFT,BUILDING_AGE
0,-0.826573,1.599627
1,1.320099,-1.076033
2,-1.034245,-0.346308
3,1.033731,-0.184146
4,-0.341276,0.464498
...,...,...
7098,-1.714097,2.005030
7099,1.125543,-1.238194
7100,0.506900,0.302337
7101,-1.300939,0.626660


In [110]:
df

,PRT_ID,AREA,INT_SQFT,DATE_SALE,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,SALE_COND,PARK_FACIL,...,SALE_DAY_NAME,SALE_QUARTER,BUILD_YEAR,BUILD_MONTH,BUILD_MONTH_NAME,BUILD_DAY,BUILD_DAY_OF_WEEK,BUILD_DAY_NAME,BUILD_QUARTER,BUILDING_AGE
0,P03210,Karapakkam,1004,2011-05-04,131,1.0,1.0,3,AbNormal,Yes,...,Wednesday,2,1967,5,May,15,0,Monday,2,44
1,P09411,Anna Nagar,1986,2006-12-19,26,2.0,1.0,5,AbNormal,No,...,Tuesday,4,1995,12,December,22,4,Friday,4,11
2,P01812,Adayar,909,2012-02-04,70,1.0,1.0,3,AbNormal,Yes,...,Saturday,1,1992,2,February,9,6,Sunday,1,20
3,P05346,Velachery,1855,2010-03-13,14,3.0,2.0,5,Family,No,...,Saturday,1,1988,3,March,18,4,Friday,1,22
4,P06210,Karapakkam,1226,2009-10-05,84,1.0,1.0,3,AbNormal,Yes,...,Monday,4,1979,10,October,13,5,Saturday,4,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,P03834,Karapakkam,598,2011-01-03,51,1.0,1.0,2,AdjLand,No,...,Monday,1,1962,1,January,15,0,Monday,1,49
7099,P10000,Velachery,1897,2004-04-08,52,3.0,2.0,5,Family,Yes,...,Thursday,2,1995,4,April,11,1,Tuesday,2,9
7100,P09594,Velachery,1614,2006-08-25,152,2.0,1.0,4,Normal Sale,No,...,Friday,3,1978,9,September,1,4,Friday,3,28
7101,P06508,Karapakkam,787,2009-08-03,40,1.0,1.0,2,Partial,Yes,...,Monday,3,1977,8,August,11,3,Thursday,3,32


In [170]:
pd.concat([X_cat_En,X_cat_En1,X_num_en],axis=1)

,AREA,SALE_COND,PARK_FACIL,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE,N_BEDROOM,N_BATHROOM,N_ROOM,INT_SQFT,BUILDING_AGE
0,6,2,0,0,0,1,5,1.0,1.0,3,-0.826573,1.599627
1,1,2,1,0,0,0,2,2.0,1.0,5,1.320099,-1.076033
2,5,2,0,0,3,0,1,1.0,1.0,3,-1.034245,-0.346308
3,3,3,1,1,1,1,3,3.0,2.0,5,1.033731,-0.184146
4,6,2,0,1,0,0,4,1.0,1.0,3,-0.341276,0.464498
...,...,...,...,...,...,...,...,...,...,...,...,...
7098,6,0,1,1,3,2,0,1.0,1.0,2,-1.714097,2.005030
7099,3,3,0,1,2,2,2,3.0,2.0,5,1.125543,-1.238194
7100,3,1,1,2,2,0,3,2.0,1.0,4,0.506900,0.302337
7101,6,4,0,0,3,1,1,1.0,1.0,2,-1.300939,0.626660
